# Vaje 7: Nevronske mreže 1

## Naloga 1: Usmerjene (feedforward) nevronske mreže 

Z ukazom `pip3 install torch torchvision` si inštaliraj paket [PyTorch](https://pytorch.org/get-started/locally/), ki ga bomo uporabljali za nevronske mreže in ukazom `pip3 install tqdm` paket [tqdm](https://github.com/tqdm/tqdm), ki ga bomo uporabljali za izpisovanje sprotnih rezultatov


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transform
from torchvision.datasets import MNIST
from tqdm import tqdm

Naložimo podatkovno množico. Uporabljali bomo slike števk, za katere bomo poskusili napovedati, katera števka je na sliki.

In [ ]:
batch_size = 32

train_set = MNIST('../Podatki/', train=True, download=True, 
                  transform=transform.Compose([transform.ToTensor(), transform.Normalize((0.1307,), (0.3081,))]))

test_set = MNIST('../Podatki/', train=False, download=True, 
                 transform=transform.Compose([transform.ToTensor(), transform.Normalize((0.1307,), (0.3081,))]))

Poglejmo si, kako naši podatki zgledajo

In [ ]:
# Prvi učni primer pretvorimo v numpy array in ga izrišemo
first_image = train_set[0][0].numpy()[0, :, :]
plt.imshow(first_image, cmap="Greys")
plt.show()

# Vhodni podatki v usmerjeno nevronsko mrežo bodo vektorji. Zato sliko pretvorimo v vektor (lepimo vrstice eno za drugo). Za lepše viden izris ta vektor 100-krat skopiramo 
plt.imshow(np.repeat(first_image.reshape((28*28, 1)).T, repeats=100, axis=0), cmap="Greys")
plt.show()

# Izpišemo ciljno vrednost prvega učnega podatka
print(f"Label {train_set[0][1]}")

Sestavimo napovedni model, ga natrenirajmo in potestiramo

In [ ]:
# Definirajmo našo nevronsko mrežo
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # Definiramo objekt, ki bo sliko spremenil v vektor
        self.flatten = nn.Flatten()
        # Definiramo sloje naše nevronske mreže
        self.linear_relu_stack = nn.Sequential(
            # Polno povezan sloj z vhodno dimenzijo 28*28 in izhodno dimenzijo 64
            nn.Linear(28*28, 64),
            # Aktivacijska funkcija ReLu
            nn.ReLU(),
            nn.Linear(64, 20),
            nn.ReLU(),
            nn.Linear(20, 10)
        )
        # Definiramo še objekt, ki bo izhod pretvoril v "verjetnosti" (pozitivne vrednosti, ki se šeštejejo v 1)
        # parameter dim=1 nam to naredi po drugi (1-ti) dimenziji. Vsak podatek po prvi (0-ti) dimenziji je ena slika v skupini (batchu)
        self.softmax = nn.Softmax(dim=1)
        

    def forward(self, x):
        # Vhodno sliko pretvorimo v vektor
        x = self.flatten(x)
        # Vektor pošljemo čez vse sloje in aktivacijske vrednosti
        logits = self.linear_relu_stack(x)
        # Izhod iz nevronske mreže pretvorimo v "verjetnosti" za vsako ciljno vrednost
        return self.softmax(logits)

In [ ]:
# Naredimo funkcijo, ki bo natrenirala naš model
def train(epochs, model, trainset, batch_size, learning_rate=0.01):
    
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    
    # Zagotovimo, da bo model v načinu treniranja, kjer se računajo gradienti in so aktivni vsi sloji (med evalvacijo niso nujno vsi aktivni, npr. Dropout, BatchNormalization, ...)
    model.train()
    
    # Definiramo naš optimizator. V našem primeru bo to stohastični gradientni spust (SGD) z learning rate-om 0.01
    # Lahko bi uporabili tudi Adam naprimer
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    
    # Definiramo našo funkcijo izgube (loss). V našem primeru bo to prečna entropija.
    criterion = nn.CrossEntropyLoss()
    
    # Gremo čež epohe
    for epoch in range(epochs):

        running_loss = 0.0
        # Uporabimo funkcijo tqdm.tqdm, ki nam bo lepše, v realnem času izpisovala napredek učenja
        with tqdm(total=len(trainloader)*batch_size, desc=f'Training - Epoch: {epoch + 1}/{epochs}', unit='chunks') as prog_bar:
            # Gremo čez vse podatke v skupinah po batch_size z trainloaderjem, v našem primeru je to 32
            for i, data in enumerate(trainloader, 0):
                # Podatke razpakiramo v vhode in izhode (labele)
                inputs, labels = data

                # Počistimo (resetiramo) gradiente v podatkih
                optimizer.zero_grad()

                # Vhodne podatke spustimo čez model, ta nam vrne matriko, v kateri se vsaka vrstica sešteje v 1 (zaradi Softmax sloja)
                outputs = model(inputs)
                # Izračunamo izgubo
                loss = criterion(outputs, labels)
                # Naredimo vzvratno razširanje napake (backpropagation)
                loss.backward()
                # Naredimo en korak optimizacije
                optimizer.step()

                # Dodamo izgubo k naši vsoti izgube. S funkcijo detach poskrbimo, da ne prištejemo (in si tako shranimo) tudi gradienta
                # s funkcijo item() pa da se le vrednost in ne celoten vektor
                running_loss += loss.detach().item()

                # Posodobimo vrednosti funkcije tqdm.tqdm. Vsoto dosedanje izgube delimo s številom skupin, ki smo jih že obdelali
                prog_bar.set_postfix(**{'loss': (running_loss) / (i+1)})
                # Posodobimo progress bar
                prog_bar.update(batch_size)

    print('Finished Training')

In [ ]:
def test(model, testset, batch_size):
    
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    # Model damo v način evalvacije. Tu se gradienti ne računajo in da se nekateri sloji deaktivirajo
    model.eval()
    test_loss = 0
    correct = 0
    counter = 0
    
    # Dodatno poskrbimo, da vektorji ne vsebujejo gradientov
    with torch.no_grad():
        with tqdm(total=len(testloader)*batch_size, desc=f'Testing', unit='chunks') as prog_bar:
            for i, data in enumerate(testloader, 0):
                inputs, labels = data
                output = model(inputs)
                test_loss += criterion(output, labels).detach().item()
                # Izberemo indekse mesta z najvišjo vrednostjo ("verjetnostjo")
                pred = output.data.max(1, keepdim=True)[1]
                # Prištejemo število primerov, kjer smo zadeli pravilno števko
                correct += pred.eq(labels.data.view_as(pred)).sum()
                prog_bar.update(batch_size)
                counter += 1
    
    print(f'Test set: Avg. loss: {test_loss/counter}, Correct predictions: {correct}/{len(testloader.dataset)}')

In [ ]:
torch.manual_seed(42)
model = NeuralNetwork()
epochs = 5

print("Accuracy on the test set before training")
print(end="")
test(model, test_set, batch_size)
print()

train(epochs, model, train_set, batch_size)
print()

print("Accuracy on the test set after training the model")
test(model, test_set, batch_size)

1.a: Preveri število parametrov v natreniranem modelu. Pomagaj si z modelovo funkcijo `parameters()` in funkcijo tenzorja `numel()`, ki izpiše število vrednosti znotraj tenzorja. Kako se število parametrov primerja z dosedaj videnimi napovednimi modeli in kaj predstavljajo vmesne vrednosti na sodih mestih?

1.b: Spremeni zgornjo nevronsko mrežo tako, da ji dodaš še en polno-povezan sloj in spremeni srednjo aktivacijsko funkcijo iz ReLu v Tanh in preveri njeno točnost.

1.c: Preiskusi točnost nevronske mreže z različnimi velikosti skupin (batch-ev), optimizatorji (SGD in Adam) in različnimi hitrostmi učenja (learning rate).

In [ ]:
from torch.utils.data import Subset
train_subset = Subset(train_set, range(5000))

## Naloga 2: Konvolucijska nevronska mreža

2.a: S pomočjo 2D konvolucijskega sloja ([`nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)) in maxpool sloja ([`nn.MaxPool2d`](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)) sestavi konvolucijsko nevronsko mrežo in jo potestiraj na testni množici.

2.b: Izpiši število parametrov te nevronske mreže. Kako se število parametrov razlikuje od števila parametrov usmerjene nevronske mreže?

2.c: Modelu dodaj še en konvolucijski sloj in maxpooling sloj, ga natreniraj, preveri njegovo točnost na testni množici in preveri koliko parametrov ima. Opaziš kaj nenavadnega?

## Naloga 3: Eksperimenti z nevronskimi mrežami

3.a: Poigraj se z naslednjimi parametri, da dobiš čim boljši model:
- število konvolucijskih slojev v mreži
- število polno-povezanih slojev v mreži
- velikost jedra konvolucijskega sloja
- velikost jedra maxpool sloja
- število nevronov v polno povezanih slojih
- število izhodnih kanalov konvolucijskega sloja
- hitrost učenja
- optimizator (Adam)

3.b: Opiši, kako bi z nevronsko mrežo sestavil model linearne regresije, ki se parametrov nauči iterativno in ne eksaktno.